<a href="https://colab.research.google.com/github/daichira-gif/daichira/blob/main/hojichar_basedon_tsuji_san_kohmoto_sancode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U hojichar==0.9.0
!pip show hojichar
!pip install regex

# import re
import regex as re
import hojichar
import json
import random
from multiprocessing import cpu_count

# ダミーデータの生成
num_samples = 100000
dummy_texts = ["こんにちは山本君、これはテストデータです。佐藤さん", "田中くん、ダミーテキストのサンプルです。", "HojiCharを使用して前処理を行います。"]
with open("your_text.jsonl", "w") as f:
    for _ in range(num_samples):
        text = random.choice(dummy_texts)
        f.write(json.dumps({"text": text}) + "\n")

# 敬称別漢字除去フィルター
class RemoveKanjiBeforeHonorific(hojichar.core.filter_interface.Filter):
    def apply(self, document):
        text = document.text
        # 文の先頭にある名前と敬称（と可能な句読点）を削除
        text = re.sub(r'^\p{Script=Han}+(?:さん|くん|君)[、。]?', '', text)
        # 文中の名前と敬称を削除し、その後に来る可能な句読点は残す
        text = re.sub(r'(\P{IsHan}+)\p{Script=Han}+(?:さん|くん|君)([、。]?)', r'\1\2', text)
        document.text = text
        return document

# Composeオブジェクトの定義
cleaner = hojichar.Compose([
    hojichar.document_filters.JSONLoader(),
    hojichar.document_filters.DocumentNormalizer(),
    RemoveKanjiBeforeHonorific(),
    hojichar.document_filters.JSONDumper(),
])

# 大量のJSON Linesファイルを並列に処理
input_file = "your_text.jsonl"
with open(input_file, 'r') as file:
    lines = file.readlines()

input_doc_iter = (hojichar.Document(line) for line in lines)

num_cores = cpu_count()
with hojichar.Parallel(cleaner, num_jobs=num_cores) as pfilter:
    out_doc_iter = pfilter.imap_apply(input_doc_iter)
    with open("your_processed_text.jsonl", "w") as fp:
        for doc in out_doc_iter:
            fp.write(doc.text + "\n")

# 結果の表示
num_display = 10
with open("your_processed_text.jsonl", "r") as f:
    for i, line in enumerate(f):
        if i >= num_display:
            break
        print(line.strip())

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 667.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.6/67.6 kB 2.4 MB/s eta 0:00:00
Name: hojichar
Version: 0.9.0
Summary: Text preprocessing management system.
Home-page: https://github.com/HojiChar/HojiChar
Author: kenta.shinzato
Author-email: hoppiece@gmail.com
License: Apache-2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: mmh3, numpy, tqdm
Required-by: 
{"text": "HojiCharを使用して前処理を行います。"}
{"text": "こんにちは、これはテストデータです。"}
{"text": "こんにちは、これはテストデータです。"}
{"text": "ダミーテキストのサンプルです。"}
{"text": "HojiCharを使用して前処理を行います。"}
{"text": "こんにちは、これはテストデータです。"}
{"text": "ダミーテキストのサンプルです。"}
{"text": "ダミーテキストのサンプルです。"}
{"text": "ダミーテキストのサンプルです。"}
{"text": "HojiCharを使用して前処理を行います。"}


メールアドレスと電話番号も削除

In [ ]:

import regex  as re
import hojichar
import json
import random
from multiprocessing import cpu_count


# ダミーデータの生成
num_samples = 100000
dummy_texts = [
    "こんにちは山本君、これはテストデータです。電話番号は09056781234です。佐藤さんのメールアドレスをご存じですか？",
    "田中くんのメールアドレスはjtwmdagp@gmail.comです。",
    "HojiCharを使用して前処理を行います。",
]

with open("your_text.jsonl", "w") as f:
    for _ in range(num_samples):
        text = random.choice(dummy_texts)
        f.write(json.dumps({"text": text}) + "\n")


class RemoveKanjiBeforeHonorificAndPII(hojichar.core.filter_interface.Filter):
    """
    「さん」や「くん」などの敬称の直前にある漢字と敬称、電話番号、メールアドレスを削除するフィルター

    Args:
        patterns (dict): 削除対象のパターン
    """

    def __init__(self):
        super().__init__()
        self.patterns = {
            "kanji_and_honorific": r'(?P<kanji>\P{IsHan}+)\p{Script=Han}+(?P<honorific>さん|くん|君)(\p{Script=Hiragana}{1,2})?([、。]?)',
            "phone_number": r'\d{11}|\d{3}-\d{4}-\d{4}',
            "email": r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}',
        }

    def apply(self, document):
        """
        処理対象のテキストに対して、敬称の直前の漢字と敬称、電話番号、メールアドレスを削除する処理を実行します。

        Args:
            document (hojichar.document.Document): 処理対象のドキュメント

        Returns:
            hojichar.document.Document: 処理後のドキュメント
        """
        text = document.text

        # 文の先頭にある名前と敬称（と可能な句読点）を削除
        text = re.sub(r'^\p{Script=Han}+(?:さん|くん|君)[、。]?', '', text)

        # 文中の名前と敬称を削除し、その後に来る可能な句読点は残す
        text = re.sub(r'(\P{IsHan}+)\p{Script=Han}+(?:さん|くん|君)([、。]?)', r'\1\2', text)

        # 電話番号、メールアドレスを削除
        for pattern_name, pattern in self.patterns.items():
            text = re.sub(pattern, r'', text)

        document.text = text
        return document


cleaner = hojichar.Compose([
    hojichar.document_filters.JSONLoader(),
    hojichar.document_filters.DocumentNormalizer(),
    RemoveKanjiBeforeHonorificAndPII(),
    hojichar.document_filters.JSONDumper(),
])

# 大量のJSON Linesファイルを並列に処理
input_file = "your_text.jsonl"
with open(input_file, 'r') as file:
    lines = file.readlines()

input_doc_iter = (hojichar.Document(line) for line in lines)

num_cores = cpu_count()
with hojichar.Parallel(cleaner, num_jobs=num_cores) as pfilter:
    out_doc_iter = pfilter.imap_apply(input_doc_iter)
    with open("your_processed_text.jsonl", "w") as fp:
        for doc in out_doc_iter:
            fp.write(doc.text + "\n")

# 結果の表示
num_display = 10
with open("your_processed_text.jsonl", "r") as f:
    for i, line in enumerate(f):
        if i >= num_display:
            break
        print(line.strip())

{"text": "のメールアドレスはです。"}
{"text": "のメールアドレスはです。"}
{"text": "HojiCharを使用して前処理を行います。"}
{"text": "HojiCharを使用して前処理を行います。"}
{"text": "のメールアドレスはです。"}
{"text": "こんにちは、これはテストデータです。電話番号はです。のメールアドレスをご存じですか?"}
{"text": "のメールアドレスはです。"}
{"text": "こんにちは、これはテストデータです。電話番号はです。のメールアドレスをご存じですか?"}
{"text": "こんにちは、これはテストデータです。電話番号はです。のメールアドレスをご存じですか?"}
{"text": "こんにちは、これはテストデータです。電話番号はです。のメールアドレスをご存じですか?"}
